In [ ]:
# default_exp timeseries.core

In [ ]:
#hide
#export
import pandas as pd
from fastai.data.external import *
from fastcore.all import *
from pathlib import PosixPath
from fastcore.test import *
from fastai.tabular.all import *
import fastai
from fastai.tabular.core import _maybe_expand
from fastrenewables.tabular.core import *
from fastrenewables.utils import *

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
files = glob.glob('../data/*.h5')
dfs = read_files(files)

In [ ]:
def get_data(splits=None):
    return TabularRenewables(pd.concat(dfs), 
                       pre_process=FilterInconsistentSamplesPerDay, 
                       splits=splits,
                       cat_names="TaskID",
                       cont_names = [ 'WindSpeed58m', 'WindSpeed60m'],
                       y_names="PowerGeneration",
                      )

In [ ]:
#export
def _convert_tabular_pandas_to_timeseries(
        to, 
        ts_x_length=None, 
        ts_y_length=None, 
        y_timeseries_offset=1,
        step_size=None
    ):
        if ts_x_length is None and ts_y_length is None and step_size is None:
            timeseries_length = get_samples_per_day(to.items)
            ts_x_length = timeseries_length
            ts_y_length = timeseries_length
            step_size = timeseries_length
        elif ts_x_length is not None or ts_y_length is not None or step_size is not None:
            raise ValueError("Either none or all values must be provided for (ts_x_length, ts_y_length, step_size)")
        
        len_data = len(to.xs)
        has_cats = len(to.cat_names) > 0
        is_cont_output = contains_instance(list(to.procs), RegressionSetup)
        
        if len_data % ts_x_length != 0:
            raise Exception("Length of data is not dividable by length of a timeseries sequence.")
        
        max_samples = len_data - y_timeseries_offset - ts_y_length

        samples_tensor = max_samples // step_size + 1

        conts = torch.zeros((samples_tensor, to.conts.shape[1], ts_x_length)).float()
        ys = torch.zeros((samples_tensor, to.ys.shape[1], ts_y_length))

        cats = None
        if has_cats:
            cats = torch.zeros((samples_tensor, to.cats.shape[1], ts_x_length)).long()
        else:
            cats = torch.empty(((samples_tensor, to.cats.shape[1], ts_x_length)))
            
        for sample_id, i in enumerate(range(0, max_samples + 1, step_size)):
            start_x = i
            end_x = start_x + ts_x_length
            start_y = i + y_timeseries_offset
            end_y = start_y + ts_y_length

            conts[sample_id, :, :] = tensor(
                to.conts[start_x:end_x].values.transpose().reshape(-1, ts_x_length)
            )
            if has_cats:
                cats[sample_id, :, :] = tensor(
                    to.cats[start_x:end_x]
                    .values.transpose()
                    .reshape(-1, ts_x_length)
                ).long()

            ys[sample_id, :, :] = tensor(
                to.ys.iloc[start_y:end_y]
                .values.transpose()
                .reshape(-1, ts_y_length)
            )
            
        if is_cont_output:
            print("is cont")
            ys = ys.float()
        else:
            print("is cat")
            ys = ys.long()

        return cats, conts, ys

In [ ]:
to = get_data()
cats, conts, ys = _convert_tabular_pandas_to_timeseries(to)

In [ ]:
# TODO:
#   check for types of cats and ys
#   check for exception when incorrect size of dataframe
#   to.items = to.items[:-2]
# _convert_tabular_pandas_to_timeseries(to)

In [ ]:
#export

def _check_categoricals(conts, cats, ys, batch_first=True, sequence_last=True):
    # If selected, drop all categorical columns which do not have a constant value for each time series
    # individually.
    if batch_first and sequence_last:
        for i in range(cats.shape[1]):
            keep_indexes = []
            for j in range(cats.shape[0]):
                if (cats[j, i, :] - cats[j, i, 0]).sum() == 0:
                    keep_indexes += [j]

            n_dropped = cats.shape[0] - len(keep_indexes)
            if n_dropped > 0:
                warnings.warn(
                    f"Dropped {n_dropped} elements due to inconsistent categoricals in a sequence."
                )
            conts = conts[keep_indexes, :, :]
            cats = cats[keep_indexes, :, :]
            ys = ys[keep_indexes, :, :]
    else:
        raise NotImplementedError(
            f"Drop inconsistent categoricals is not implemented for batch_first {self.batch_first} and sequence_last {self.sequence_last}"
        )

In [ ]:
to = get_data()
cats, conts, ys = _convert_tabular_pandas_to_timeseries(to)
cats[1,0,1] = 1
_check_categoricals(conts, cats, ys)

In [ ]:
#export

def _adjust_ts_and_batch(data, batch_first, sequence_last):
    """
    Swap the dimensions of the given Tensor.
    Parameters
    ----------
    data : pytorch.Tensor
        Three dimensional Tensor whose dimensions are to be swapped. Expectes data of the dimension (batch, features, sequence length).
    batch_first : bool
        determines whether the first dimension of the resulting Tensors should denote the batch.
    sequence_last : bool
        determines whether the last dimension of the resulting Tensors should denote the sequence length.

    Returns
    -------
    data : pytorch.Tensor
        input tensor with swapped dimensions.
    """

    if batch_first and sequence_last:
        # batch, feature, seq -> batch, seq, feature
        pass
    elif batch_first and not sequence_last:
        # batch, feature, seq -> batch, seq, feature
        data = data.permute(0, 2, 1)
    elif not batch_first and not sequence_last:
        # batch, feature, seq -> seq, batch, feature
        data = data.permute(2, 0, 1)

    return data


In [ ]:
#export

def _convert_to_batch_ts_feature_view(data, batch_first, sequence_last):
    """
    Converts the data to the followong dimension (batch, sequence length, features).

    Parameters
    ----------
    data : pytorch.Tensor
        three dimensional Tensor whose dimensions are to be swapped.
    batch_first : bool
        determines whether the first dimension of the resulting Tensors denotes the batch.
    sequence_last : bool
        determines whether the last dimension of the resulting Tensors denotes the sequence length.

    Returns
    -------
    data : pytorch.Tensor
        input tensor with dimensions to be swapped.
    """

    if batch_first and sequence_last:
        # batch, feature, seq -> batch, seq, feature
        data = data.permute(0, 2, 1)
    elif not batch_first and not sequence_last:
        # seq, batch, feature -> batch, seq, feature
        data = data.permute(1, 0, 2)
    elif not batch_first and sequence_last:
        # feature, batch, seq -> batch, seq, feature
        data = data.permute(1, 2, 0)

    return data

In [ ]:
#export
class TimeseriesTransform(Transform, FilteredBase):
    def __init__(
        self,
        to: TabularRenewables,
        timeseries_length: int = 24,
        batch_first: bool = True,
        sequence_last: bool = True,
        drop_inconsistent_cats: bool = True,
        y_timeseries_offset=0,
        step_size=24,
        splits=None
    ):
        """

        Parameters
        ----------
        to : TabularRenewables
            input dataset, has to be set up before.
        timeseries_length : integer
            amount of elements in each time series used for the neural network.
        batch_first : bool
            determines whether the first dimension of the resulting Tensors denoted the batch.
        sequence_last : bool
            determines whether the last dimension of the resulting Tensors denoted the sequence length.
        """
        
        self.to = to
        self.has_cat = len(to.cat_names) > 0
        
        if getattr(to, "splits", None) is not None:
            warnings.warn("Splitting in TabularRenewables is currentl supported. \
                          Otherwise a correct splitting for timeseries is not assured. \
                          Fallback to complete data.")
            
        self.timeseries_length = timeseries_length
        self.step_size = step_size
        self.batch_first = batch_first
        self.sequence_last = sequence_last
        
        self.cats, self.conts, self.ys = _convert_tabular_pandas_to_timeseries(to)

        self._adjust_to_required_timeseries_representation()
        
        if splits is None:
            self.split = range(0,len(self))
        else:
            # TODO: check if it is already an object?
            splits = splits(self)
            self.split = splits

    def _adjust_to_required_timeseries_representation(self):
        self.conts = _adjust_ts_and_batch(
            self.conts, self.batch_first, self.sequence_last
        )
        self.ys = _adjust_ts_and_batch(
            self.ys, self.batch_first, self.sequence_last
        )
        if self.has_cat:
            self.cats = _adjust_ts_and_batch(
                self.cats, self.batch_first, self.sequence_last
            )
            
    # required for splitting of train and eval
    def subset(self, i): 
        # ToDo: index=None, batch_first=True, sequence_last=True
#         split = slice(0,self.split) if i==0 else slice(self.split,len(self))
#         split = self.split if i==0 else slice(self.split,len(self))
        if len(self.split) == 2:
            return TimeseriesDataset(self._data_by_split(self.split[i]), self._all_names())
        elif i == 0:
            return TimeseriesDataset(self._data_by_split(self.split), self._all_names())
        else:
            return None
        
    @property
    def train(self):
        return self.subset(0)
    @property
    def valid(self):
        return self.subset(1)
    
    def _data_by_split(self, split):
        return (self.cats[split], self.conts[split], self.ys[split])
    
    def _all_names(self):
        return (self.cat_names, self.cont_names, self.y_names)
            
    @property
    def cont_names(self):
        return self.to.cont_names
    
    @property
    def cat_names(self):
        return self.to.cat_names
    
    @property
    def y_names(self):
        return self.to.y_names
    
    def __len__(self):
        """
        Return the length of the used TabularPandas.

        Returns
        -------
        integer
            amount of lines in the TabularPandas
        """
        # TODO depends on position of batch
        return self.conts.shape[0]

    def show(self, max_n=10, **kwargs):
        """
        Create a plot for a 'max_n' of input- and target time series.
        Parameters
        ----------
        max_n : interger
            amount of samples that are to be plotted.
        kwargs

        Returns
        -------

        """
        # TODO: make subplots with grid
        tmp_data = TimeseriesTransform._adjust_ts_and_batch(
            self.xs, batch_first=True, sequence_last=True
        )

        for idx in range(self.xs.shape[1]):
            plt.plot(self.xs[0:max_n, idx, :].reshape(-1, 1))
            plt.title(f"Feature {self.tp.cont_names[idx]}")
            plt.show()

        for idx in range(self.ys.shape[1]):
            plt.plot(self.ys[0:max_n, idx, :].reshape(-1, 1))
            plt.title(f"Target {self.tp.y_names[idx]}")
            plt.show()

    def _all_names(self):
        return [self.to.cat_names, self.to.cont_names, self.to.y_names]
    


In [ ]:
#export
class TimeseriesDataset(fastuple):
    "A dataset from a `TimeseriesTransform` object"
#     TODO add index
    def __init__(self, data, names, index=None, batch_first=True, sequence_last=True):
        self.cats, self.conts, self.ys = data
        self.cat_names, self.cont_names, self.y_names = names
        if not batch_first or not sequence_last:
            raise NotImplementedError

    def __getitem__(self, idx):
        idx = idx[0]
        return self.cats[idx:idx+self.bs], self.conts[idx:idx+self.bs], self.ys[idx:idx+self.bs]

    def __len__(self): return len(self.conts)
    
    def _as_df(self, max_n):
        df_cont = pd.DataFrame(data=self.conts[:max_n].reshape(-1,self.conts.shape[1]), columns=self.cont_names)
        df_cat = pd.DataFrame(data=self.cats[:max_n].reshape(-1,self.cats.shape[1]), columns=self.cat_names)
        df_y = pd.DataFrame(data=self.ys[:max_n].reshape(-1,self.ys.shape[1]), columns=self.y_names)
        return pd.concat([df_cont, df_cat, df_y], axis=1)
    
    def __repr__(self):
        represenation = f"{self.cat_names}:\n{self.cats}\n\n{self.cont_names}:\n{self.conts}\n\n{self.y_names}:\n{self.ys}\n\n"
        return represenation
    
    def show(self, max_n=10, **kwargs): 
        display_df(self._as_df(max_n))
        
    def show_batch(self, max_n=10, **kwargs): 
        self.show()

In [ ]:
#export
class TimeSeriesDataLoaders(DataLoaders):
    def __init__(self, to, bs=64, val_bs=None, shuffle_train=True, device='cpu', **kwargs):
        train_ds = to.train
        valid_ds = to.valid
        val_bs = bs if val_bs is None else val_bs
        train = TabDataLoader(train_ds, bs=bs, shuffle=shuffle_train, device=device, **kwargs)
        valid = TabDataLoader(valid_ds, bs=val_bs, shuffle=False, device=device, **kwargs)
        super().__init__(train, valid, device=device, **kwargs)

In [ ]:
to = get_data()

In [ ]:
tt = TimeseriesTransform(to, splits=RandomSplitter())

is cont


In [ ]:
dls = TabDataLoaders(tt)

In [ ]:
dl = TabDataLoader(tt.train, bs=16)
cats,conts, ys = dl.one_batch()
cats.shape, conts.shape, ys.shape

(torch.Size([16, 1, 24]), torch.Size([16, 2, 24]), torch.Size([16, 1, 24]))

In [ ]:
dl = TabDataLoader(tt.valid, bs=16, shuffle=True)
cats,conts, ys = dl.one_batch()
cats.shape, conts.shape, ys.shape

(torch.Size([16, 1, 24]), torch.Size([16, 2, 24]), torch.Size([16, 1, 24]))

In [ ]:
# TODO: adapt based on tabular core dataloader, add splits in 